### Figures 3k, S4f, S4m

In [ ]:
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, BoundaryNorm
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
meta = pd.read_csv("HD-OV 100.csv", index_col=False)
sampleName = meta['sampleName']

In [ ]:

cnv_cluster = pd.read_csv('cnv_clu5_th2.csv', index_col=0)
cnv_cluster['cnv_cluster'] = cnv_cluster.astype(int).astype(str)

mapping_dict = {'1': 'CIN-low', '2': 'Chr8q-amp', '3': 'Chr20-amp', '4': 'Chr18-amp', '5': 'CIN-moderate'}
cnv_cluster['cnv_cluster'] = cnv_cluster['cnv_cluster'].replace(mapping_dict)

In [ ]:
for subdirectory in sampleName: 
    # read the adata file
    adata = sc.read_h5ad(f"{subdirectory}/adata_infercnv.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)

    # obtain cnv states
    adata.obs.index = adata.obs.index + '_' + subdirectory
    adata.obs['cnv_cluster'] = cnv_cluster

    # delete NA
    adata_filtered = adata[~adata.obs['cnv_cluster'].isnull()]

    # create the color dictionary
    color_dict = {'Chr8q-amp': '#fbb395', 'Chr20-amp': '#ffdc7e', 'Chr18-amp': '#7dd1a9', 'CIN-moderate': '#4ca6a3', 'CIN-low': '#69619c'}

    # create a legend with only the actual colors used
    actual_colors = [key for key in color_dict.keys() if key in adata_filtered.obs['cnv_cluster'].unique()]
    legend_patches = [mpatches.Patch(color=color_dict[key], label=key) for key in actual_colors]
    
    # plot the spatial distribution of the cnv states
    sc.pl.spatial(adata_filtered, color='cnv_cluster', img_key=None, show=False, size=1.4, palette=color_dict)
    legend = plt.legend(handles=legend_patches, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.savefig(f"../{subdirectory}_cluster_spatial.png", format="png", bbox_inches='tight', bbox_extra_artists=[legend])
    

### Figures 3f, S4m

In [ ]:
for sample in sampleName: 
    # read the adata file
     adata = sc.read_h5ad(f"{sample}/adata_infercnv.h5ad")
     adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
     adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
     adata.obs['array_row'] = adata.obs['array_row'].astype(float)
     adata.obs['array_col'] = adata.obs['array_col'].astype(float)

     colors = ['#781c68', '#b79ad8',  '#ffce94', '#ff9c2b']
     cmap = ListedColormap(colors)

     min_value = np.min(adata.obs['cnv_score'])
     max_value = np.max(adata.obs['cnv_score'])
     q1 = np.percentile(adata[adata.obs['annotations'] == 'Fibroblast'].obs['cnv_score'], 95)
     q3 = (max_value + q1)/2
     q2 = (q3 + q1)/2

     bounds = [min_value, q1, q2, q3, max_value]
     norm = BoundaryNorm(bounds, cmap.N)

     interval_labels = ['Bottom', 'Lower-Mid', 'Upper-Mid', 'Top']
     adata.obs['cnv_score_interval'] = pd.cut(adata.obs['cnv_score'], bins=bounds, labels=interval_labels, include_lowest=True)

     sc.pl.spatial(adata, color='cnv_score_interval', palette=colors, size=1.4, img=None, img_key=None, show=False)
     ax = plt.gca()
     ax.axis('off')
     ax.set_title('')
     plt.savefig(f"./sp_cnv_state/{sample}_cnv_score.png", format="png", dpi=800, bbox_inches='tight')
     plt.close()
